# About

In [1]:
# Set the repository root as a working directory 
%cd ..

d:\MLOps\Hands_on\DVC\DVC


In [25]:
import yaml

dvc_yaml = 'dvc.yaml'

with open(dvc_yaml, "r") as f:
    config = yaml.safe_load(f)


config['stages']['evaluate'].update({
  'metrics': [{'reports/metrics.json':{'cache':False}}],
  'outs': ['reports/confusion_matrix.png']
})

with open(dvc_yaml, "w") as f:
    yaml.safe_dump(config, f)
print('Update dvc.yaml')

Update dvc.yaml


In [41]:
!dvc repro 

Running stage 'data_load':
> python src/stages/data_load.py --config=params.yaml
2024-10-14 09:37:11,117 — DATA_LOAD — INFO — Get dataset
2024-10-14 09:37:11,123 — DATA_LOAD — INFO — Save raw data

Running stage 'featurize':
> python src/stages/featurize.py --config=params.yaml
2024-10-14 09:37:12,129 — FEATURIZE — INFO — Load raw data
2024-10-14 09:37:12,134 — FEATURIZE — INFO — Extract features
2024-10-14 09:37:12,136 — FEATURIZE — INFO — Save features

Running stage 'data_split':
> python src/stages/data_split.py --config=params.yaml
2024-10-14 09:37:13,814 — DATA_SPLIT — INFO — Load features
2024-10-14 09:37:13,818 — DATA_SPLIT — INFO — Split features into train and test sets
2024-10-14 09:37:13,819 — DATA_SPLIT — INFO — Save train and test sets

Running stage 'train':
> python src/stages/train.py --config=params.yaml
2024-10-14 09:37:15,719 — TRAIN — INFO — Get estimator name
2024-10-14 09:37:15,719 — TRAIN — INFO — Estimator: logreg
2024-10-14 09:37:15,719 — TRAIN — INFO — Load t

In [43]:
!dvc metrics show

Path                  f1_score
reports\metrics.json  0.93056


In [42]:
!dvc metrics diff

Path                  Metric    HEAD    workspace    Change
reports\metrics.json  f1_score  -       0.93056      -


In [33]:
!git add dvc.lock
!dvc push

1 file pushed


# Simple metric tracking with DVC



In [50]:
params_yaml = 'params.yaml'

with open(params_yaml, "r") as f:
    config = yaml.safe_load(f)


config['train'].update({
  'cv': 5,
  'estimator_name': 'svm',
})

with open(params_yaml, "w") as f:
    yaml.safe_dump(config, f)
print('Update params.yaml')

Update params.yaml


In [51]:
!dvc repro 

Stage 'data_load' didn't change, skipping
Stage 'featurize' didn't change, skipping
Stage 'data_split' didn't change, skipping
Running stage 'train':
> python src/stages/train.py --config=params.yaml
2024-10-14 09:45:30,836 — TRAIN — INFO — Get estimator name
2024-10-14 09:45:30,836 — TRAIN — INFO — Estimator: svm
2024-10-14 09:45:30,836 — TRAIN — INFO — Load train dataset
2024-10-14 09:45:30,839 — TRAIN — INFO — Train model
Fitting 5 folds for each of 8 candidates, totalling 40 fits
2024-10-14 09:45:30,955 — TRAIN — INFO — Best score: 0.9644444444444445
2024-10-14 09:45:30,955 — TRAIN — INFO — Save model
Updating lock file 'dvc.lock'

Running stage 'evaluate':
> python src/stages/evaluate.py --config=params.yaml
2024-10-14 09:45:33,334 — EVALUATE — INFO — Load model
2024-10-14 09:45:33,384 — EVALUATE — INFO — Load test dataset
2024-10-14 09:45:33,386 — EVALUATE — INFO — Evaluate (build report)
2024-10-14 09:45:33,389 — EVALUATE — INFO — Save metrics
2024-10-14 09:45:33,390 — EVALUATE 

In [52]:
!dvc metrics show 

Path                  f1_score
reports\metrics.json  1.0


In [53]:
!dvc metrics diff

Path                  Metric    HEAD     workspace    Change
reports\metrics.json  f1_score  0.93056  1.0          0.06944


In [54]:
!dvc params diff

Path         Param                 HEAD    workspace
params.yaml  train.cv              3       5
params.yaml  train.estimator_name  logreg  svm


# Add plots

Artifact `reports/cm_plot.csv` represents csv format of DVC plot metrics. DVC provides building plots by plot metrics files.

To make artifact plot metrics file move `reports/cm_plot.csv` from `outs` to `plots` section (like with metrics):


`dvc.yaml`
```yaml
...
evaluate:
    cmd: python src/stages/evaluate.py --config=params.yaml
    deps:
    - data/processed/test_iris.csv
    - models/model.joblib
    - src/stages/evaluate.py
    params:
    - base
    - data_split.testset_path
    - evaluate
    - featurize.target_column
    metrics:
    - reports/metrics.json:
        cache: false
    plots:
    - reports/cm_plot.csv:
        cache: false
    outs:
    - reports/confusion_matrix.png
```

In [65]:
params_yaml = 'params.yaml'

with open(params_yaml, "r") as f:
    config = yaml.safe_load(f)


config['evaluate'].update({
  'confusion_matrix_data': 'cm_plot.csv',
})

with open(params_yaml, "w") as f:
    yaml.safe_dump(config, f)
print('Update params.yaml')

Update params.yaml


In [62]:
dvc_yaml = 'dvc.yaml'

with open(dvc_yaml, "r") as f:
    config = yaml.safe_load(f)


config['stages']['evaluate'].update({
  'plots': [{'reports/cm_plot.csv':{'cache':False}}],
  'outs': ['reports/confusion_matrix.png']
})

with open(dvc_yaml, "w") as f:
    yaml.safe_dump(config, f)
print('Update dvc.yaml')

Update dvc.yaml


In [64]:
!dvc repro -f

Running stage 'data_load':
> python src/stages/data_load.py --config=params.yaml
2024-10-14 10:19:02,823 — DATA_LOAD — INFO — Get dataset
2024-10-14 10:19:02,830 — DATA_LOAD — INFO — Save raw data

Running stage 'featurize':
> python src/stages/featurize.py --config=params.yaml
2024-10-14 10:19:03,668 — FEATURIZE — INFO — Load raw data
2024-10-14 10:19:03,671 — FEATURIZE — INFO — Extract features
2024-10-14 10:19:03,672 — FEATURIZE — INFO — Save features

Running stage 'data_split':
> python src/stages/data_split.py --config=params.yaml
2024-10-14 10:19:05,184 — DATA_SPLIT — INFO — Load features
2024-10-14 10:19:05,187 — DATA_SPLIT — INFO — Split features into train and test sets
2024-10-14 10:19:05,188 — DATA_SPLIT — INFO — Save train and test sets

Running stage 'train':
> python src/stages/train.py --config=params.yaml
2024-10-14 10:19:07,174 — TRAIN — INFO — Get estimator name
2024-10-14 10:19:07,174 — TRAIN — INFO — Estimator: svm
2024-10-14 10:19:07,174 — TRAIN — INFO — Load trai

In [66]:
!dvc plots show

file:///D:/MLOps/Hands_on/DVC/DVC/dvc_plots/index.html


In [67]:
!dvc plots diff

file:///D:/MLOps/Hands_on/DVC/DVC/dvc_plots/index.html


# Experiment 1 - Tune LogisticRegression

## Create and run exeperiment


Create experiment and change parameter - add options for **C** hyperparamter in section `train:estimators:logreg:param_grid`:



In [80]:
!dvc exp run -n exp1-tune-logreg -f \
    --set-param train.estimators.logreg.param_grid.C=[0.1,1.0,10] \
    --set-param train.estimators.logreg.param_grid.max_iter=[200]

Reproducing experiment 'exp1-tune-logreg'
Running stage 'data_load':
> python src/stages/data_load.py --config=params.yaml
2024-10-14 10:39:38,356 — DATA_LOAD — INFO — Get dataset
2024-10-14 10:39:38,361 — DATA_LOAD — INFO — Save raw data

Running stage 'featurize':
> python src/stages/featurize.py --config=params.yaml
2024-10-14 10:39:39,237 — FEATURIZE — INFO — Load raw data
2024-10-14 10:39:39,240 — FEATURIZE — INFO — Extract features
2024-10-14 10:39:39,242 — FEATURIZE — INFO — Save features

Running stage 'data_split':
> python src/stages/data_split.py --config=params.yaml
2024-10-14 10:39:40,888 — DATA_SPLIT — INFO — Load features
2024-10-14 10:39:40,891 — DATA_SPLIT — INFO — Split features into train and test sets
2024-10-14 10:39:40,893 — DATA_SPLIT — INFO — Save train and test sets

Running stage 'train':
> python src/stages/train.py --config=params.yaml
2024-10-14 10:39:42,573 — TRAIN — INFO — Get estimator name
2024-10-14 10:39:42,574 — TRAIN — INFO — Estimator: logreg
2024-


as result you should have LogisticRegression config:

```yaml
...
train:
  cv: 3
  estimator_name: logreg

  estimators:

    logreg: # sklearn.linear_model.LogisticRegression
      param_grid: # params of GridSearchCV constructor
        C:
        - 0.1
        - 1.0
        - 10
        max_iter: [100]
        solver: [lbfgs]
        multi_class: [multinomial]
...

```


## Commit experiment results

In [82]:
!git add .
!git commit -m "Experiment 1 with LogisticRegression hyperparameters"
!git tag -a "exp1_tune_logreg" -m "Experiment 1 with LogisticRegression hyperparameters"
!dvc push

[main b91bb98] Experiment 1 with LogisticRegression hyperparameters
 2 files changed, 12 insertions(+), 12 deletions(-)


fatal: tag 'exp1_tune_logreg' already exists


1 file pushed


## Show metrics 

In [83]:
!dvc metrics show


Path                  f1_score
reports\metrics.json  1.0


# Experiment 2 - Use SVM

## Create and run experiment

in section `train` change:
    - `estimator_name`
    - change `estimators:svm:param_grid:C`



In [84]:
!dvc exp run -n exp2-svm \
    --set-param train.estimator_name=svm \
    --set-param train.estimators.svm.param_grid.C=[0.1,1.0,10]

Reproducing experiment 'exp2-svm'
Stage 'data_load' didn't change, skipping
Stage 'featurize' didn't change, skipping
Stage 'data_split' didn't change, skipping
Running stage 'train':
> python src/stages/train.py --config=params.yaml
2024-10-14 10:41:04,151 — TRAIN — INFO — Get estimator name
2024-10-14 10:41:04,151 — TRAIN — INFO — Estimator: svm
2024-10-14 10:41:04,151 — TRAIN — INFO — Load train dataset
2024-10-14 10:41:04,153 — TRAIN — INFO — Train model
Fitting 3 folds for each of 12 candidates, totalling 36 fits
2024-10-14 10:41:04,265 — TRAIN — INFO — Best score: 0.9580911025968497
2024-10-14 10:41:04,265 — TRAIN — INFO — Save model
Updating lock file 'dvc.lock'

Running stage 'evaluate':
> python src/stages/evaluate.py --config=params.yaml
2024-10-14 10:41:06,600 — EVALUATE — INFO — Load model
2024-10-14 10:41:06,652 — EVALUATE — INFO — Load test dataset
2024-10-14 10:41:06,654 — EVALUATE — INFO — Evaluate (build report)
2024-10-14 10:41:06,665 — EVALUATE — INFO — Save metrics




as result you should have SVC config:

```yaml
...
train:
  cv: 3
  estimator_name: svm
  estimators:
        
    svm: # sklearn.svm.SVC
      param_grid:
        C:
        - 0.1
        - 1.0
        - 10
        kernel: [rbf, linear]
        gamma: [scale]
        degree: [3, 5]
...
```


## Commit experiment results


In [85]:
!git add .
!git commit -m "Experiment 2 with SVM estimator"
!git tag -a "exp2_svm" -m "Experiment 2 with SVM estimator"
!dvc push

[main f2ecaaa] Experiment 2 with SVM estimator
 2 files changed, 10 insertions(+), 8 deletions(-)
1 file pushed


## Show metrics 



In [86]:
!dvc metrics show

Path                  f1_score
reports\metrics.json  1.0


# Compare experiments

1. List experiments:

```bash
dvc exp list --all
```
```
# output
940fee6:                                                              
        exp2-svm
step-6:
        exp1-tune-logreg

```

In [117]:
!dvc exp list --all-commits

10a6c91:
	e5feb59 [exp1-tune-logreg]
b91bb98:
	35b52eb [exp2-svm]
exp1_tune_logreg:
	bfcd65a [exp1-tune-logreg]




2. Compare current experiments`:


In [119]:
!dvc exp diff 10a6c91 exp1_tune_logreg


Path         Param                                 10a6c91    exp1_tune_logreg    Change
params.yaml  evaluate.confusion_matrix_data        -          cm_plot.csv         diff not supported
params.yaml  train.cv                              3          5                   2
params.yaml  train.estimator_name                  logreg     svm                 diff not supported
params.yaml  train.estimators.logreg.param_grid.C  [0.001]    [0.1, 1.0, 10]      diff not supported


In [120]:
!dvc exp show

 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

# Push changes to remote repository

```bash
git push origin step-6
```

# DVC Studio


## View creation

1. Open [DVC Studio](https://studio.iterative.ai/) and sing up/sing in.

2. Create **View**: click **Add a view**

3. Select repository

## Work with View

1. Enter `View`

2. Navigate the branch `step-6`

3. Select one of commits and click `Show plots` - you'll see plots built by `DVC Studio` for selected commit

4. Select any **two commits** to compare it and click `Compare` - `DVS Studio` will build comparison table